In [1]:
import cv2
import numpy as np
from os import listdir
import json
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
import pickle
with open('/Users/sparky/Documents/DroneOpencv/coords.pickle', 'rb') as file:
    gps_lst = pickle.load(file)
pickup_cnt = 0
dropoff_cnt = 0
pickup_dict = {}
dropoff_dict = {}

In [2]:
def click_event(event, x, y, flags, params):
    global pickup_cnt, dropoff_cnt, pickup_dict, dropoff_dict
    # checking for left mouse clicks
    if event == cv2.EVENT_FLAG_CTRLKEY:
        print("Pickups: {}".format(pickup_dict))
        print("Dropoffs: {}".format(dropoff_dict))
        return 1
    if event == cv2.EVENT_LBUTTONDOWN:
        print("Checking!")
        pickup_cnt += 1
        # displaying the coordinates
        # on the image window
        cv2.circle(imReg, (x,y), 10, (255,255,255), 2)
        cv2.circle(imReg, (x,y), 3, (0,0,255), 3)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(imReg, "{:.8} {:.8}".format(gps_lst[y][x][0],gps_lst[y][x][1]), (x,y), font,
                    1, (0, 0, 255), 2)
        print("Pickup #{}: {}, {}".format(pickup_cnt, gps_lst[y][x][0], gps_lst[y][x][1]))
        pickup_dict['pickup {}'.format(pickup_cnt)] = (gps_lst[y][x][0], gps_lst[y][x][1])
        cv2.imshow('image', imReg)
        cv2.waitKey(1)
        print('ok')
    elif event == cv2.EVENT_RBUTTONDOWN:
        dropoff_cnt += 1
        cv2.circle(imReg, (x,y), 10, (255,255,255), 2)
        cv2.circle(imReg, (x,y), 3, (0,255,0), 3)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(imReg, "{:.8} {:.8}".format(gps_lst[y][x][0],gps_lst[y][x][1]), (x,y), font,
                    1, (0, 255, 0), 2)
        print("Dropoff #{}: {}, {}".format(dropoff_cnt, gps_lst[y][x][0], gps_lst[y][x][1]))
        dropoff_dict['dropoff {}'.format(dropoff_cnt)] = (gps_lst[y][x][0], gps_lst[y][x][1])
        cv2.imshow('image', imReg)
        cv2.waitKey(1)

In [3]:
def alignImages(im1, im2):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
    matches = list(matches)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    #cv2.imshow('matches', imMatches)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #cv2.imwrite("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
    
    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

In [4]:
def alignImages(sift, keypoints2, descriptors2, im1, im2):
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    

    # Detect SIFT features and compute descriptors.
    


    keypoints1, descriptors1 = sift.detectAndCompute(im1Gray, None)
    print("Test")
    # Match features.
    FLAN_INDEX_KDTREE = 0
    index_params = dict (algorithm = FLAN_INDEX_KDTREE, trees=5)
    search_params = dict (checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    good_matches = []
    good_matches1 = []
    matches = flann.knnMatch (descriptors1, descriptors2, k=2)
    print("Test2")
    for m1, m2 in matches:
      if m1.distance < 0.6 * m2.distance:
        good_matches.append([m1])
        good_matches1.append(m1)
    # Sort matches by score
    flann_matches =cv2.drawMatchesKnn(im1, keypoints1, im2, keypoints2, good_matches, None, flags=2)
    cv2.imshow('image', flann_matches)
    cv2.waitKey(0)
    query_pts = np.float32([keypoints1[m.queryIdx] 
                .pt for m in good_matches1]).reshape(-1, 1, 2) 
 
    train_pts = np.float32([keypoints2[m.trainIdx] 
                    .pt for m in good_matches1]).reshape(-1, 1, 2) 

    h, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0) 
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    
    
    cv2.imshow("image", im1Reg) 
    cv2.waitKey(0)
    # Draw top matches
    #imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    #cv2.imshow('matches', imMatches)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #cv2.imwrite("matches.jpg", imMatches)

    # Extract location of good matches
    #points1 = np.zeros((len(matches), 2), dtype=np.float32)
    #points2 = np.zeros((len(matches), 2), dtype=np.float32)

    #for i, match in enumerate(matches):
    #    points1[i, :] = keypoints1[match.queryIdx].pt
    #    points2[i, :] = keypoints2[match.trainIdx].pt
    
    # Find homography
    #h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    #height, width, channels = im2.shape
    #im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

In [ ]:
if __name__ == '__main__':
    # Read reference image
    refFilename = "/Users/sparky/Documents/DroneOpencv/samplePics/image_0.jpeg"
    print("Reading reference image : ", refFilename)
    imBase = cv2.imread(refFilename, cv2.IMREAD_COLOR)
    sift = cv2.SIFT_create(900)
    im2Gray = cv2.cvtColor(imBase, cv2.COLOR_BGR2GRAY)
    keypoints2, descriptors2 = sift.detectAndCompute(im2Gray, None)
    for imFilename in listdir("/Users/sparky/Documents/DroneOpencv/samplePics/"):
        if imFilename.endswith('.jpeg'):
            imFilename = "/Users/sparky/Documents/DroneOpencv/samplePics/" + imFilename
            # Read image to be aligned
            print("Reading image to align : ", imFilename)
            imActual = cv2.imread(imFilename, cv2.IMREAD_COLOR)

            print("Aligning images ...")
            # Registered image will be resotred in imReg.
            # The estimated homography will be stored in h.
            imReg, h = alignImages(sift, keypoints2, descriptors2, imActual, imBase)
            added_image = cv2.addWeighted(imReg,0.5,imBase,0.5,0)
            #print("Showing added image : ")
            #cv2.imshow('added', added_image)
            #cv2.waitKey(0)
            #cv2.imwrite(outFilename, imReg)
            cv2.imshow('image', added_image)
            cv2.setMouseCallback('image', click_event)
            while 1:
                if cv2.waitKey(1) == ord('q'):
                    print("q!")
                    break
            # Print estimated homography
    with open('/Users/sparky/Documents/DroneOpencv/POI_GPS.json', 'w') as file:
                json.dump({'pickups':pickup_dict, 'dropoffs':dropoff_dict}, file)
    cv2.destroyAllWindows()
    print("Estimated homography : \n",  h)

Reading reference image :  /Users/sparky/Documents/DroneOpencv/samplePics/image_0.jpeg
Reading image to align :  /Users/sparky/Documents/DroneOpencv/samplePics/image_6.jpeg
Aligning images ...
Test
Test2
q!
Reading image to align :  /Users/sparky/Documents/DroneOpencv/samplePics/image_7.jpeg
Aligning images ...
Test
Test2
q!
Reading image to align :  /Users/sparky/Documents/DroneOpencv/samplePics/image_0.jpeg
Aligning images ...
Test
Test2
q!
Reading image to align :  /Users/sparky/Documents/DroneOpencv/samplePics/image_1.jpeg
Aligning images ...
Test
Test2
q!
Reading image to align :  /Users/sparky/Documents/DroneOpencv/samplePics/image_2.jpeg
Aligning images ...
Test
Test2


In [ ]:
with open('/Users/sparky/Documents/DroneOpencv/POI_GPS.json', 'r') as file:
    coords = json.load(file)
for coord in coords['pickups'].keys():
    with open('/Users/sparky/Documents/DroneOpencv/{}.waypoints'.format(coord), 'w') as file:
        file.writelines('QGC WPL 110 \n')
        file.writelines('0\t1\t0\t16\t0\t0\t0\t0\t33.520752\t-84.581770\t3144.245759\t1\n')
        file.writelines('1\t0\t3\t22\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t12.192000\t1\n')
        file.writelines('2\t0\t3\t16\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t{}\t{}\t12.192000\t1\n'.format(coords['pickups'][coord][0], coords['pickups'][coord][1]))
        file.writelines('3\t0\t3\t21\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.000000\t1\n')
        file.writelines('4\t0\t3\t211\t0.00000000\t1.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.000000')
for coord in coords['dropoffs'].keys():
    with open('/Users/sparky/Documents/DroneOpencv/{}.waypoints'.format(coord), 'w') as file:
        file.writelines('QGC WPL 110 \n')
        file.writelines('0\t1\t0\t16\t0\t0\t0\t0\t33.520752\t-84.581770\t3144.245759\t1\n')
        file.writelines('1\t0\t3\t22\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t12.192000\t1\n')
        file.writelines('2\t0\t3\t16\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t{}\t{}\t12.192000\t1\n'.format(coords['dropoffs'][coord][0], coords['dropoffs'][coord][1]))
        file.writelines('3\t0\t3\t21\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.000000\t1\n')
        file.writelines('4\t0\t3\t211\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.00000000\t0.000000')

In [10]:
cv2.__version__

'4.6.0'

In [19]:
[thing for thing in dir(cv2) if "xfeat" in thing]

[]